# CatsAndDogs Feature Extraction

**Objective:** Extract the features using ResNet50.

## About ResNet50

* [Intro to ResNet50](https://www.kaggle.com/keras/resnet50/home)
* [Paper: Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)
* **The input size of the image is 224x224xRGB**
* **Since we are NOT using the FC network of ResNet50, the flattened output from the final pooling layer is 2048.**
  Refer to the above mentioned paper for the network architecture.

## Load ResNet50 pretrained weights

In [1]:
from keras.applications import ResNet50

Using Theano backend.


In [2]:
#?ResNet50

In [3]:
# load the ResNet50 network
print('Loading ResNet50 network weights')
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

Loading ResNet50 network weights


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

bn3a_branch2a (BatchNormalizati (None, None, None, 1 512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_11 (Activation)      (None, None, None, 1 0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, None, None, 1 147584      activation_11[0][0]              
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, None, None, 1 512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_12 (Activation)      (None, None, None, 1 0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_bran

res4a_branch2a (Conv2D)         (None, None, None, 2 131328      activation_22[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, None, None, 2 1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_23 (Activation)      (None, None, None, 2 0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, None, None, 2 590080      activation_23[0][0]              
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, None, None, 2 1024        res4a_branch2b[0][0]             
__________________________________________________________________________________________________
activation

bn4e_branch2b (BatchNormalizati (None, None, None, 2 1024        res4e_branch2b[0][0]             
__________________________________________________________________________________________________
activation_36 (Activation)      (None, None, None, 2 0           bn4e_branch2b[0][0]              
__________________________________________________________________________________________________
res4e_branch2c (Conv2D)         (None, None, None, 1 263168      activation_36[0][0]              
__________________________________________________________________________________________________
bn4e_branch2c (BatchNormalizati (None, None, None, 1 4096        res4e_branch2c[0][0]             
__________________________________________________________________________________________________
add_12 (Add)                    (None, None, None, 1 0           bn4e_branch2c[0][0]              
                                                                 activation_34[0][0]              
__________

bn5c_branch2c (BatchNormalizati (None, None, None, 2 8192        res5c_branch2c[0][0]             
__________________________________________________________________________________________________
add_16 (Add)                    (None, None, None, 2 0           bn5c_branch2c[0][0]              
                                                                 activation_46[0][0]              
__________________________________________________________________________________________________
activation_49 (Activation)      (None, None, None, 2 0           add_16[0][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           activation_49[0][0]              
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
__________________________________________________________________________________________________


## Get the Training Image Names

In [5]:
import pathlib
import random

In [6]:
ImageDir = './datasets/train'

# recursively go over the datset folder and get the file names.
ImageList = list(pathlib.Path(ImageDir).rglob('*.jpg'))
random.shuffle(ImageList)
print(len(ImageList))

25000


In [7]:
print(ImageList[0:4])

[PosixPath('datasets/train/dog.7265.jpg'), PosixPath('datasets/train/dog.3432.jpg'), PosixPath('datasets/train/cat.10089.jpg'), PosixPath('datasets/train/dog.12092.jpg')]


### Encode the Labels

In [8]:
import os

In [9]:
train_labels = []
for imagePath in ImageList:
    file_name = imagePath.as_posix()
    class_label = file_name.split(os.path.sep)[-1].split('.')[0]
    #print(file_name, ' ', class_label)
    train_labels.append(class_label)


In [10]:
print(len(train_labels))

25000


In [11]:
train_labels[0:3]

['dog', 'dog', 'cat']

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)

In [14]:
le.classes_

array(['cat', 'dog'], dtype='<U3')

In [15]:
train_labels[0:3]

array([1, 1, 0])

In [16]:
len(train_labels)

25000

### Configuration Settings

In [17]:
# total number of images to process

# for initial testing the scripting
#NUM_IMAGES = 100

# for complete dataset, turn this ON.
NUM_IMAGES = len(ImageList)

### buffer info.
BUFF_SIZE = 1000

## batch
BATCH_SIZE=20

## DataSetWriter

In [18]:
import h5py
import os

In [19]:
db = h5py.File('./output/cats_and_dogs_features.hdf5', 'w')

In [20]:
type(db)

h5py._hl.files.File

In [21]:
#create the dictionary like dataset entries

# the output from the max pool layer from ResNet50

features = db.create_dataset('features', (NUM_IMAGES, 2048), dtype='float')
labels   = db.create_dataset('labels', (NUM_IMAGES,), dtype='int')

In [22]:
feature_idx = 0

In [23]:
## Create the buffer for both the features and labels, so that we can flush them to disk when it is full.

#create buffer dictionary
buffer = { 'features': [], 'labels': [] }
# index to the list - features
feature_idx = 0

### Buffer Utilities

In [24]:
# write buffer utilities

## add the features and labels from buffer -> db
def flush_the_buffer():
    global feature_idx
    global buffer
    global features
    global labels
    
    to_idx = feature_idx + len(buffer['features'])
    features[feature_idx:to_idx] = buffer['features']
    labels[feature_idx:to_idx]   = buffer['labels']
    
    #update the feature idx
    feature_idx = to_idx
    
    #reset the buffer
    buffer = { 'features': [], 'labels': [] }
    return

In [25]:
## add the features and labels to the buffer
def add_to_buffer(feature_entries, labels):
    global buffer
    
    buffer['features'].extend(feature_entries)
    buffer['labels'].extend(labels)
    
    if len(buffer['features']) >= BUFF_SIZE :
        flush_the_buffer()
    return

In [26]:
## close the db
def close_the_database():
    global buffer
    global db
    
    if len(buffer['features']) > 0:
        flush_the_buffer()
        
    db.close()
    return

## Extract Features

* Load the image and resize them to 224x224 for ResNet50
* Preprocess the image for ResNet

* Use model.predict() method

In [27]:
import numpy as np
import cv2
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

In [28]:
## resize them to one size
def preprocess_image(img, width, height, interpolation=cv2.INTER_AREA):
    return( cv2.resize(img, (width, height), interpolation))

In [29]:
# go over all the images to extract the features in batches
for i in np.arange(0, NUM_IMAGES, BATCH_SIZE):
    #
    # process them in batches
    #
    batchImageList = ImageList[i: i+BATCH_SIZE]
    batchLabelsList = train_labels[i: i+BATCH_SIZE]
    batchImages = []
    #print(i)
    
    # loop over each image in the batchImage List
    for (j, imagePath) in enumerate(batchImageList):
        fileName = imagePath.as_posix()
        # load the image
        img = cv2.imread(fileName)
        img = preprocess_image(img, 224, 224)
        
        # convert to array
        img = img_to_array(img)
        
        ##preprocess the input for the ResNet architecture.
        ## it needs in four dimensions, so expand
        img = np.expand_dims(img, axis=0)
        img = imagenet_utils.preprocess_input(img)
        
        # add image to the batch
        batchImages.append(img)
        
    ## now pass the images thru ResNet50 network architecture and
    batchImages = np.vstack(batchImages)
    extracted_features = model.predict(batchImages, batch_size=BATCH_SIZE)
    
    #print('extracted_features.shape :', extracted_features.shape)
    ## get the extracted features for each image after the max pooling layer.
    ## note that, the size after the max pooling layer is 2048 from the ResNet50 arch.
    #
    extracted_features = extracted_features.reshape((extracted_features.shape[0], 2048))
    
    ## add the features to the HDF5 dataset
    add_to_buffer(extracted_features, batchLabelsList)
    
## close the HDF5 dataset
close_the_database()

In [30]:
print('Done!')

Done!
